In [20]:
from opendbc.car import structs
from opendbc.car.hyundai.values import CAR, HyundaiFlags
from opendbc.car.hyundai.fingerprints import FW_VERSIONS

#TEST_PLATFORMS = set(CAR.with_flags(HyundaiFlags.EV | HyundaiFlags.HYBRID)) - set(CAR.with_flags(HyundaiFlags.CANFD))  # CAN hybrid and electric vehicles only
TEST_PLATFORMS = [CAR.HYUNDAI_IONIQ_PHEV, ]

print(f"Found {len(TEST_PLATFORMS)} qualifying vehicles:")
for platform in TEST_PLATFORMS:
  print(f"  {platform}")

Found 1 qualifying vehicles:
  HYUNDAI_IONIQ_PHEV


In [29]:
import random

from openpilot.tools.lib.logreader import LogReader
from openpilot.tools.lib.comma_car_segments import get_comma_car_segments_database
from opendbc.car.hyundai.values import CAR

database = get_comma_car_segments_database()
TEST_SEGMENTS = []

MAX_SEGS_PER_PLATFORM = 40  # TODO: Increase this to search more segments

print("Collecting segments from commaCarSegments dataset:")
for platform in TEST_PLATFORMS:
  assert(platform in database)
  #if platform not in database:
  #  print(f"Skipping platform: {platform}, no data available")
  #  continue
  
  all_segments = database[platform]

  NUM_SEGMENTS = min(len(all_segments), MAX_SEGS_PER_PLATFORM)

  print(f"Got {len(all_segments)} segments for platform {platform}, sampling {NUM_SEGMENTS} segments")

  TEST_SEGMENTS.extend(random.sample(all_segments, NUM_SEGMENTS))

print(f"Collected {len(TEST_SEGMENTS)} segments for analysis")


Got 1115 segments for platform HYUNDAI_IONIQ_PHEV, sampling 40 segments
Collected 40 segments for analysis


In [31]:
import copy
import matplotlib.pyplot as plt
import numpy as np

from opendbc.can.parser import CANParser
from opendbc.car import Bus
from opendbc.car.hyundai.values import DBC

from openpilot.selfdrive.pandad import can_capnp_to_list
from openpilot.tools.lib.logreader import LogReader

message_names = ["ELECT_GEAR"]

for segment in TEST_SEGMENTS:
  lr = LogReader(segment)
  CP = lr.first("carParams")
  if CP is None:
    continue

  can_msgs = [msg for msg in lr if msg.which() == "can"]
  parser_messages = []
  for name in message_names:
    parser_messages.append((name, 0))
  cp = CANParser(DBC[platform][Bus.pt], parser_messages, 0)

  parsed_message_history = []
  examples = []

  for msg in can_msgs:
    cp.update_strings(can_capnp_to_list([msg.as_builder().to_bytes()]))
    parsed_message_history.append(copy.copy(cp.vl))

  print(f"Analyzing segment {segment:<44} for {CP.carFingerprint}")
  for name in message_names:
    if parsed_message_history[0][name]["Elect_Gear_Shifter"] != 0:  # Message is present for this segment
      gear_prev = parsed_message_history[0][name]["Elect_Gear_Shifter"]
      if gear_prev == 4.0:
        print(f"  {name:<15} gear={gear_prev}")
      for i, parsed_messages in enumerate(parsed_message_history):
        gear = parsed_messages[name]["Elect_Gear_Shifter"]
        if gear != gear_prev:
          print(f"  *** Signal transition found! ***")
          examples.append(i)
        gear_prev = gear

print(f"Analysis finished")


Analyzing segment 8fd9d7b55c053355/2023-11-22--14-15-12/22/s   for HYUNDAI IONIQ PHEV 2020
Analyzing segment f78ea8bac6cf6c9d/2023-12-07--07-31-30/18/s   for HYUNDAI IONIQ PHEV 2020
Analyzing segment 37f27727e9056e1c/2023-12-12--16-27-58/4/s    for HYUNDAI IONIQ PHEV 2020
Analyzing segment 37f27727e9056e1c/2023-12-14--15-16-25/64/s   for HYUNDAI IONIQ PHEV 2020
Analyzing segment c447f3983314c453/2024-01-15--16-09-23/19/s   for HYUNDAI IONIQ PHEV 2020
Analyzing segment 8fd9d7b55c053355/2023-12-06--13-29-24/20/s   for HYUNDAI IONIQ PHEV 2020
Analyzing segment 74f6ab69c669f8c5/2023-11-06--14-32-39/3/s    for HYUNDAI IONIQ PHEV 2020
Analyzing segment 74f6ab69c669f8c5/2023-12-05--12-30-06/10/s   for HYUNDAI IONIQ PHEV 2020
Analyzing segment 8fd9d7b55c053355/2023-11-22--18-28-57/11/s   for HYUNDAI IONIQ PHEV 2020
Analyzing segment 78fdfacf4a805561/2023-12-12--17-31-17/76/s   for HYUNDAI IONIQ PHEV 2020
Analyzing segment 37f27727e9056e1c/2024-01-25--14-37-27/83/s   for HYUNDAI IONIQ PHEV 2020